Milestone2


In [259]:
import kagglehub
from kagglehub import KaggleDatasetAdapter
import pandas as pd
import numpy as np

# kagglehub.login()


Dataset available at: https://www.kaggle.com/datasets/tonygordonjr/football-match-statistics-and-more/data?fbclid=IwZXh0bgNhZW0CMTEAAR27HPyyqC9nUioeOqJzVzznFoIH3SNY0xcdZ_4dDxEpDOoRnZgcaXdTTRE_aem_0dX5wF1-vk6WmlQWZMnQ2g

In [260]:
# !kaggle datasets download -d tonygordonjr/football-match-statistics-and-more
# !unzip /content/football-match-statistics-and-more.zip -d ./

In [261]:
teams_clean_col = ['fixture_id', 'team_id', 'team_name', 'fouls','yellow_cards', 'red_cards']
stats_clean_col = ['fixture_id', 'fixture_date', 'fixture_referee', 'league_id', 'league_name', 'teams_home_id', 'teams_home_name', 'teams_away_id', 'teams_away_name']
players_clean_col = ['fixture_id', 'team_id', 'team_name', 'player_id', 'player_name', 'fouls_committed', 'yellow_cards', 'red_cards', 'game_minutes']

fix_teams_clean = pd.read_csv("fixture_stats_teams_clean.csv",usecols=teams_clean_col)
fix_clean = pd.read_csv("fixture_stats_clean.csv", usecols=stats_clean_col)
fix_players_clean = pd.read_csv("fixture_stats_players_clean.csv",usecols=players_clean_col)

In [262]:
# team ids are actual identifiers (not like player_ids)
fix_teams_clean.groupby(['team_id', 'team_name']).size().reset_index().team_name.value_counts()

,count
team_name,
Bay FC,1
Belgium,1
France,1
Robin Hood,1
Al Riyadh,1
...,...
Colombia,1
Uruguay,1
Brazil,1


In [263]:
# fixture_date added to the players statistics for further data preparation purposes
fix_players_clean_og = fix_players_clean.merge(fix_clean[['fixture_id', 'fixture_date']], on='fixture_id', how='left')
fix_players_clean = fix_players_clean_og.copy()

In [264]:
fix_players_clean['total_cards'] = fix_players_clean.apply(lambda x: x['yellow_cards'] + x['red_cards'] * 2 if x['yellow_cards'] != 2 else 3, axis=1)
# fix_players_clean.sort_values('red_cards', ascending=False).head()

fix_players_clean.drop(['yellow_cards', 'red_cards'], axis=1, inplace=True)
fix_total_cards = fix_players_clean.groupby(['fixture_id', 'team_id'])['total_cards'].sum().reset_index()

In [265]:
fix_total_cards = fix_total_cards.merge(fix_clean[['fixture_id', 'fixture_date']], on='fixture_id', how='left')

In [266]:
fix_total_cards

,fixture_id,team_id,total_cards,fixture_date
0,65,33,2,2018-08-10 19:00:00 UTC
1,65,46,1,2018-08-10 19:00:00 UTC
2,66,34,2,2018-08-11 11:30:00 UTC
3,66,47,2,2018-08-11 11:30:00 UTC
4,67,35,1,2018-08-11 14:00:00 UTC
...,...,...,...,...
64871,1316662,1602,2,2024-12-01 00:30:00 UTC
64872,1316663,1595,4,2024-12-01 03:00:00 UTC
64873,1316663,1605,0,2024-12-01 03:00:00 UTC
64874,1318600,1602,2,2024-12-07 21:00:00 UTC


In [267]:
# Sort by team and fixture_date to ensure past matches come first
fix_total_cards = fix_total_cards.sort_values(by=["team_id", "fixture_date"])

# Compute past average total cards for each team
fix_total_cards["avg_total_cards"] = (
    fix_total_cards.groupby("team_id")["total_cards"]
    .expanding()
    .mean()
    .shift()  # Shift to exclude current row from the average
    .reset_index(level=0, drop=True)
)

# Fill NaN with 0 (or keep as NaN if you prefer)
fix_total_cards["avg_total_cards"] = fix_total_cards["avg_total_cards"].fillna(0)

In [268]:
fix_total_cards.sort_values(by=["team_id", "fixture_date"]).head(10)

,fixture_id,team_id,total_cards,fixture_date,avg_total_cards
19868,152593,1,1,2016-06-13 19:00:00 UTC,0.000000
19892,152605,1,1,2016-06-18 13:00:00 UTC,1.000000
19920,152619,1,2,2016-06-22 19:00:00 UTC,1.000000
19836,152574,1,3,2016-06-26 19:00:00 UTC,1.333333
19842,152578,1,2,2016-07-01 19:00:00 UTC,1.750000
16950,135789,1,3,2018-06-18 15:00:00 UTC,1.800000
16978,135803,1,0,2018-06-23 12:00:00 UTC,2.000000
17018,135823,1,2,2018-06-28 18:00:00 UTC,1.714286
16910,135766,1,0,2018-07-02 18:00:00 UTC,1.750000
16916,135770,1,2,2018-07-06 18:00:00 UTC,1.555556


In [269]:
cards_tmp = fix_clean.merge(fix_total_cards.drop("fixture_date", axis=1), on="fixture_id", how="inner")
cards_tmp = cards_tmp[cards_tmp["team_id"] == cards_tmp["teams_home_id"]]
cards_tmp = cards_tmp.rename(columns={"total_cards": "home_total_cards_value", "avg_total_cards": "avg_home_total_cards"})
cards_tmp = cards_tmp.drop(columns=["team_id"])

cards_tmp = cards_tmp.merge(fix_total_cards.drop("fixture_date", axis=1), on="fixture_id", how="inner")
cards_tmp = cards_tmp[cards_tmp["team_id"] == cards_tmp["teams_away_id"]]
cards_tmp = cards_tmp.rename(columns={"total_cards": "away_total_cards_value", "avg_total_cards": "avg_away_total_cards"})
cards_tmp = cards_tmp.drop(columns=["team_id"])

cards_tmp["total_cards_value"] = cards_tmp["home_total_cards_value"] + cards_tmp["away_total_cards_value"]

fix_total_cards = cards_tmp

In [270]:
fix_players_clean.drop(fix_players_clean[fix_players_clean["game_minutes"].isna()].index, inplace=True)

fix_players_clean["game_minutes"].isna().sum()

np.int64(0)

In [271]:
fix_total_cards

,fixture_id,fixture_date,fixture_referee,league_id,league_name,teams_home_id,teams_home_name,teams_away_id,teams_away_name,home_total_cards_value,avg_home_total_cards,away_total_cards_value,avg_away_total_cards,total_cards_value
0,566789,2020-07-06 18:30:00 UTC,"Felix Brych, Germany",78,Bundesliga,180,FC Heidenheim,162,Werder Bremen,2,2.000000,2,2.122807,4
3,566788,2020-07-02 18:30:00 UTC,F. Zwayer,78,Bundesliga,162,Werder Bremen,180,FC Heidenheim,5,2.105882,2,1.794118,7
4,9822,2017-10-01 10:00:00 UTC,"Alfonso Alvarez Izquierdo, Spain",140,La Liga,548,Real Sociedad,543,Real Betis,3,2.726190,3,2.890244,6
6,203592,2016-02-27 21:05:00 UTC,"Alberto Undiano, Spain",140,La Liga,548,Real Sociedad,535,Malaga,4,3.160000,5,3.250000,9
8,203530,2016-01-16 21:05:00 UTC,"Juan Martinez Munuera, Spain",140,La Liga,548,Real Sociedad,544,Deportivo La Coruna,1,3.473684,3,2.631579,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
64867,132923,2019-10-11 18:45:00 UTC,O. Hațegan,4,Euro Championship,777,Türkiye,778,Albania,3,2.000000,5,2.666667,8
64869,1299137,2024-09-26 16:45:00 UTC,"Benoît Bastien, France",3,UEFA Europa League,611,Fenerbahce,1393,Union St. Gilloise,5,2.822222,4,2.000000,9
64870,350229,2016-09-29 19:05:00 UTC,"Jonas Eriksson, Sweden",3,UEFA Europa League,611,Fenerbahce,209,Feyenoord,2,3.272727,3,1.000000,5
64872,971763,2023-02-16 17:45:00 UTC,I. Peljto,3,UEFA Europa League,550,Shakhtar Donetsk,94,Rennes,3,2.157143,3,1.986971,6


In [272]:
fix_total_fouls = fix_teams_clean[["fixture_id", "team_id", "fouls"]]
fix_total_fouls = fix_total_fouls.merge(fix_clean[['fixture_id', 'fixture_date']], on='fixture_id', how='left')

In [273]:
# Sort by team and fixture_date to ensure past matches come first
fix_total_fouls = fix_total_fouls.sort_values(by=["team_id", "fixture_date"])

# Compute past average total cards for each team
fix_total_fouls["avg_total_fouls"] = (
    fix_total_fouls.groupby("team_id")["fouls"]
    .expanding()
    .mean()
    .shift()  # Shift to exclude current row from the average
    .reset_index(level=0, drop=True)
)

# Fill NaN with 0 (or keep as NaN if you prefer)
fix_total_fouls["avg_total_fouls"] = fix_total_fouls["avg_total_fouls"].fillna(0)

In [274]:
fix_total_fouls.sort_values(by=["team_id", "fixture_date"]).head(10)

,fixture_id,team_id,fouls,fixture_date,avg_total_fouls
345,152593,1,10,2016-06-13 19:00:00 UTC,0.000000
55421,152605,1,8,2016-06-18 13:00:00 UTC,10.000000
19845,152619,1,14,2016-06-22 19:00:00 UTC,9.000000
61918,152574,1,10,2016-06-26 19:00:00 UTC,10.666667
19840,152578,1,8,2016-07-01 19:00:00 UTC,10.500000
55424,135789,1,17,2018-06-18 15:00:00 UTC,10.000000
64402,135803,1,12,2018-06-23 12:00:00 UTC,11.166667
19841,135823,1,14,2018-06-28 18:00:00 UTC,11.285714
55426,135766,1,13,2018-07-02 18:00:00 UTC,11.625000
19843,135770,1,16,2018-07-06 18:00:00 UTC,11.777778


In [275]:
fix_total_fouls = fix_total_fouls[['fixture_id', 'team_id', 'avg_total_fouls']]

In [276]:
def calculate_avg_cards_per_90(df):
    # Sort by fixture_id to ensure chronological order
    df = df.sort_values(by=['player_name', 'fixture_date'])

    # Create new columns for storing cumulative stats
    df['cumulative_cards'] = 0
    df['cumulative_fouls'] = 0
    df['cumulative_minutes'] = 0
    df['avg_cards_per_90'] = 0.0
    df['avg_fouls_per_90'] = 0.0

    # Group by player_name and iterate to compute averages from past matches
    grouped = df.groupby('player_name')

    for player_name, player_df in grouped:
        cumulative_cards = 0
        cumulative_fouls = 0
        cumulative_minutes = 0

        for idx, row in player_df.iterrows():
            # Compute averages from past matches
            if cumulative_minutes > 0:
                avg_cards = (cumulative_cards / cumulative_minutes) * 90
                avg_fouls = (cumulative_fouls / cumulative_minutes) * 90
            else:
                avg_cards = 0
                avg_fouls = 0

            # Store calculated averages
            df.at[idx, 'avg_cards_per_90'] = avg_cards
            df.at[idx, 'avg_fouls_per_90'] = avg_fouls

            # Update cumulative stats
            cumulative_cards += row['total_cards']
            cumulative_fouls += row['fouls_committed']
            cumulative_minutes += row['game_minutes']

    return df

In [277]:
fix_players_clean_og

,fixture_id,team_id,team_name,player_id,player_name,game_minutes,fouls_committed,yellow_cards,red_cards,fixture_date
0,855753,1,Belgium,20,Axel Witsel,90.0,1,0,0,2022-11-27 13:00:00 UTC
1,135803,1,Belgium,2296,Eden Hazard,68.0,0,0,0,2018-06-23 12:00:00 UTC
2,855753,1,Belgium,1422,Jeremy Doku,NaN,0,0,0,2022-11-27 13:00:00 UTC
3,718242,1,Belgium,1422,Jeremy Doku,90.0,2,0,0,2021-07-02 19:00:00 UTC
4,132793,1,Belgium,2925,Dennis Praet,1.0,0,0,0,2019-03-24 19:45:00 UTC
...,...,...,...,...,...,...,...,...,...,...
1113412,1168555,22943,Bay FC,102337,Emily Menges,90.0,1,0,0,2024-05-25 02:00:00 UTC
1113413,1168522,22943,Bay FC,102337,Emily Menges,90.0,0,0,0,2024-04-28 02:00:00 UTC
1113414,1168513,22943,Bay FC,360130,Alexis Loera,32.0,2,0,0,2024-04-20 23:30:00 UTC
1113415,1168591,22943,Bay FC,194790,Deyna Castellanos,67.0,0,0,0,2024-07-07 02:00:00 UTC


In [278]:
fix_players_clean_copy = fix_players_clean.copy()
fix_players_clean = calculate_avg_cards_per_90(fix_players_clean_copy)

In [279]:
# keep only the 22 most played players for all the fixtures
fix_players_clean_final = fix_players_clean.groupby('fixture_id').apply(lambda x: x.nlargest(22, 'game_minutes')).reset_index(drop=True)

<ipython-input-279-99127c0f7190>:2: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  fix_players_clean_final = fix_players_clean.groupby('fixture_id').apply(lambda x: x.nlargest(22, 'game_minutes')).reset_index(drop=True)


In [280]:
top_leagues = ["La Liga", "Serie A", "Bundesliga", "Premier League"]
fix_clean = fix_clean[fix_clean["league_name"].isin(top_leagues)]

teams_to_keep = list(set(list(fix_clean.teams_home_name.unique())+list(fix_clean.teams_away_name.unique())))

print(len(fix_players_clean_final))
cleaned_players_clean = fix_players_clean_final[fix_players_clean_final['team_name'].isin(teams_to_keep)]
print(len(fix_players_clean_final))

713628
713628


In [281]:
cleaned_players_clean['player_num'] = cleaned_players_clean.groupby('fixture_id').cumcount() + 1

pivoted_fix_players_clean_final = cleaned_players_clean.pivot_table(
    index='fixture_id',
    columns='player_num',
    values=['avg_cards_per_90', 'avg_fouls_per_90']
)

pivoted_fix_players_clean_final.columns = [
    f'player_{num}_avg_cards_per_90' if stat == 'avg_cards_per_90'
    else f'player_{num}_avg_fouls_per_90'
    for stat, num in pivoted_fix_players_clean_final.columns
]

pivoted_fix_players_clean_final.reset_index(inplace=True)
# pivoted_fix_players_clean_final

<ipython-input-281-df5b3f1d6589>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  cleaned_players_clean['player_num'] = cleaned_players_clean.groupby('fixture_id').cumcount() + 1


In [282]:
# Adding to the final data table the teams' card statistics

# for home team
merged = fix_clean.merge(fix_total_fouls, on="fixture_id", how="inner")
merged = merged[merged["team_id"] == merged["teams_home_id"]]
merged = merged.rename(columns={"avg_total_fouls": "avg_home_team_fouls"})
merged = merged.drop(columns=["team_id"])

# for away team
merged = merged.merge(fix_total_fouls, on="fixture_id", how="inner")
merged = merged[merged["team_id"] == merged["teams_away_id"]]
merged = merged.rename(columns={"avg_total_fouls": "avg_away_team_fouls"})
merged = merged.drop(columns=["team_id"])


In [283]:
# Adding to the final data table the fix_total_cards
fix_total_cards = fix_total_cards[['fixture_id', 'home_total_cards_value', 'avg_home_total_cards', 'away_total_cards_value', 'avg_away_total_cards', 'total_cards_value']]
merged = merged.merge(fix_total_cards, on="fixture_id", how="inner")

In [284]:
merged

,fixture_id,fixture_date,fixture_referee,league_id,league_name,teams_home_id,teams_home_name,teams_away_id,teams_away_name,avg_home_team_fouls,avg_away_team_fouls,home_total_cards_value,avg_home_total_cards,away_total_cards_value,avg_away_total_cards,total_cards_value
0,566789,2020-07-06 18:30:00 UTC,"Felix Brych, Germany",78,Bundesliga,180,FC Heidenheim,162,Werder Bremen,16.000000,11.228070,2,2.000000,2,2.122807,4
1,566788,2020-07-02 18:30:00 UTC,F. Zwayer,78,Bundesliga,162,Werder Bremen,180,FC Heidenheim,11.252941,12.852941,5,2.105882,2,1.794118,7
2,9822,2017-10-01 10:00:00 UTC,"Alfonso Alvarez Izquierdo, Spain",140,La Liga,548,Real Sociedad,543,Real Betis,7.880952,8.329268,3,2.726190,3,2.890244,6
3,203592,2016-02-27 21:05:00 UTC,"Alberto Undiano, Spain",140,La Liga,548,Real Sociedad,535,Malaga,0.000000,0.000000,4,3.160000,5,3.250000,9
4,203530,2016-01-16 21:05:00 UTC,"Juan Martinez Munuera, Spain",140,La Liga,548,Real Sociedad,544,Deportivo La Coruna,0.000000,0.000000,1,3.473684,3,2.631579,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12830,878055,2022-10-29 12:00:00 UTC,Carlos del Cerro,140,La Liga,723,Almeria,538,Celta Vigo,12.909091,11.682540,4,3.363636,2,2.762069,6
12831,878215,2023-04-09 16:30:00 UTC,José Sánchez,140,La Liga,723,Almeria,532,Valencia,12.407407,11.352941,3,3.037037,2,2.898462,5
12832,877967,2022-08-27 20:00:00 UTC,Jesús Gil,140,La Liga,723,Almeria,536,Sevilla,13.500000,11.258278,5,4.500000,8,2.690476,13
12833,878075,2022-11-09 18:00:00 UTC,Alejandro Muñiz,140,La Liga,723,Almeria,546,Getafe,12.692308,14.122642,9,3.230769,4,3.536000,13


In [285]:
# Needed in the final table:
#fixture referee, league_name, teams_home_name, teams_away_name, avg_home_team_fouls, avg_home_team_cards, avg_away_team_fouls, avg_away_teasm_cards, [avg_total_cards_player_{1-22}, avg_total_fouls_player_{1_22},] total_cards


In [286]:
# TODO 1 - DONE
# Meg kéne csinálni az átlagokat home és away csapatra is szabálytalanságokból, sárga és piros lapokból
# LEHET EGYSZERŰBB EGYBŐL TODO 2-VEL PRÓBÁLKOZNI!

# merged['avg_home_team_fouls'] = merged.groupby('teams_home_name')['home_team_fouls'].transform('mean')
# merged['avg_home_team_yellow_cards'] = merged.groupby('teams_home_name')['home_team_yellow_cards'].transform('mean')
# merged['avg_home_team_red_cards'] = merged.groupby('teams_home_name')['home_team_red_cards'].transform('mean')
# merged['avg_away_team_fouls'] = merged.groupby('teams_away_name')['away_team_fouls'].transform('mean')
# merged['avg_away_team_yellow_cards'] = merged.groupby('teams_away_name')['away_team_yellow_cards'].transform('mean')
# merged['avg_away_team_red_cards'] = merged.groupby('teams_away_name')['away_team_red_cards'].transform('mean')

In [287]:
merged

,fixture_id,fixture_date,fixture_referee,league_id,league_name,teams_home_id,teams_home_name,teams_away_id,teams_away_name,avg_home_team_fouls,avg_away_team_fouls,home_total_cards_value,avg_home_total_cards,away_total_cards_value,avg_away_total_cards,total_cards_value
0,566789,2020-07-06 18:30:00 UTC,"Felix Brych, Germany",78,Bundesliga,180,FC Heidenheim,162,Werder Bremen,16.000000,11.228070,2,2.000000,2,2.122807,4
1,566788,2020-07-02 18:30:00 UTC,F. Zwayer,78,Bundesliga,162,Werder Bremen,180,FC Heidenheim,11.252941,12.852941,5,2.105882,2,1.794118,7
2,9822,2017-10-01 10:00:00 UTC,"Alfonso Alvarez Izquierdo, Spain",140,La Liga,548,Real Sociedad,543,Real Betis,7.880952,8.329268,3,2.726190,3,2.890244,6
3,203592,2016-02-27 21:05:00 UTC,"Alberto Undiano, Spain",140,La Liga,548,Real Sociedad,535,Malaga,0.000000,0.000000,4,3.160000,5,3.250000,9
4,203530,2016-01-16 21:05:00 UTC,"Juan Martinez Munuera, Spain",140,La Liga,548,Real Sociedad,544,Deportivo La Coruna,0.000000,0.000000,1,3.473684,3,2.631579,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12830,878055,2022-10-29 12:00:00 UTC,Carlos del Cerro,140,La Liga,723,Almeria,538,Celta Vigo,12.909091,11.682540,4,3.363636,2,2.762069,6
12831,878215,2023-04-09 16:30:00 UTC,José Sánchez,140,La Liga,723,Almeria,532,Valencia,12.407407,11.352941,3,3.037037,2,2.898462,5
12832,877967,2022-08-27 20:00:00 UTC,Jesús Gil,140,La Liga,723,Almeria,536,Sevilla,13.500000,11.258278,5,4.500000,8,2.690476,13
12833,878075,2022-11-09 18:00:00 UTC,Alejandro Muñiz,140,La Liga,723,Almeria,546,Getafe,12.692308,14.122642,9,3.230769,4,3.536000,13


In [288]:
# TODO 2 - DONE
# Az átlagokat úgy kéne megcsinálni hogy csak az azon meccs előtti infóknak legyen az átlaga

# avgs = ['home_team_fouls', 'home_total_cards_value', 'away_team_fouls', 'away_total_cards_value']
# merged = merged.sort_values(by=["teams_home_id", "fixture_date"])
# for avg in avgs:
#     merged[f"avg_{avg}"] = merged.groupby("teams_home_id")[avg] \
#     .expanding().mean().shift(1).reset_index(level=0, drop=True).fillna(0)

# merged["avg_home_team_cards"] = merged["avg_home_team_yellow_cards"] + merged["avg_home_team_red_cards"] * 2
# merged["avg_away_team_cards"] = merged["avg_away_team_yellow_cards"] + merged["avg_away_team_red_cards"] * 2

# merged["total_cards"] = merged["home_team_yellow_cards"] + merged["away_team_yellow_cards"] + merged["home_team_red_cards"] * 2 + merged["away_team_red_cards"] * 2

# final_cols = ["fixture_id", "fixture_referee", "league_name", "teams_home_name", "teams_away_name", "avg_home_team_fouls", "avg_away_team_fouls","avg_home_team_cards","avg_away_team_cards", "total_cards"]
# merged = merged[final_cols]


In [289]:
merged[(merged["teams_home_name"] == "Manchester United") | (merged["teams_away_name"] == "Manchester United")].sort_values(by=["fixture_date"])

,fixture_id,fixture_date,fixture_referee,league_id,league_name,teams_home_id,teams_home_name,teams_away_id,teams_away_name,avg_home_team_fouls,avg_away_team_fouls,home_total_cards_value,avg_home_total_cards,away_total_cards_value,avg_away_total_cards,total_cards_value
8031,192297,2015-08-08 11:45:00 UTC,"Jonathan Moss, England",39,Premier League,33,Manchester United,47,Tottenham,12.846154,9.484536,2,1.153846,3,1.717224,5
3508,192307,2015-08-14 18:45:00 UTC,"Mike Dean, England",39,Premier League,66,Aston Villa,33,Manchester United,0.000000,0.000000,2,3.000000,2,2.000000,4
456,192317,2015-08-22 11:45:00 UTC,"Craig Pawson, England",39,Premier League,33,Manchester United,34,Newcastle,4.000000,0.000000,2,2.000000,2,3.000000,4
9288,192336,2015-08-30 15:00:00 UTC,"Martin Atkinson, England",39,Premier League,76,Swansea,33,Manchester United,0.000000,3.800000,2,2.333333,2,2.000000,4
5449,192355,2015-09-20 15:00:00 UTC,"Mark Clattenburg, England",39,Premier League,41,Southampton,33,Manchester United,5.750000,2.714286,2,2.000000,0,1.800000,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8015,1208128,2024-11-10 14:00:00 UTC,P. Bankes,39,Premier League,33,Manchester United,46,Leicester,10.377232,9.483029,0,2.031390,1,1.716146,1
470,1208138,2024-11-24 16:30:00 UTC,A. Taylor,39,Premier League,57,Ipswich,33,Manchester United,10.771186,10.374165,0,1.927966,0,2.026846,0
7945,1208148,2024-12-01 13:30:00 UTC,J. Brooks,39,Premier League,33,Manchester United,45,Everton,10.365854,10.260989,2,2.022272,3,1.916667,5
9835,1208154,2024-12-04 20:15:00 UTC,S. Barrott,39,Premier League,42,Arsenal,33,Manchester United,9.352535,10.369469,1,1.771889,3,2.022222,4


In [290]:
merged_final = merged.merge(pivoted_fix_players_clean_final, on="fixture_id", how="left")
merged_final

,fixture_id,fixture_date,fixture_referee,league_id,league_name,teams_home_id,teams_home_name,teams_away_id,teams_away_name,avg_home_team_fouls,avg_away_team_fouls,home_total_cards_value,avg_home_total_cards,away_total_cards_value,avg_away_total_cards,total_cards_value,player_1_avg_cards_per_90,player_2_avg_cards_per_90,player_3_avg_cards_per_90,player_4_avg_cards_per_90,player_5_avg_cards_per_90,player_6_avg_cards_per_90,player_7_avg_cards_per_90,player_8_avg_cards_per_90,player_9_avg_cards_per_90,player_10_avg_cards_per_90,player_11_avg_cards_per_90,player_12_avg_cards_per_90,player_13_avg_cards_per_90,player_14_avg_cards_per_90,player_15_avg_cards_per_90,player_16_avg_cards_per_90,player_17_avg_cards_per_90,player_18_avg_cards_per_90,player_19_avg_cards_per_90,player_20_avg_cards_per_90,player_21_avg_cards_per_90,player_22_avg_cards_per_90,player_1_avg_fouls_per_90,player_2_avg_fouls_per_90,player_3_avg_fouls_per_90,player_4_avg_fouls_per_90,player_5_avg_fouls_per_90,player_6_avg_fouls_per_90,player_7_avg_fouls_per_90,player_8_avg_fouls_per_90,player_9_avg_fouls_per_90,player_10_avg_fouls_per_90,player_11_avg_fouls_per_90,player_12_avg_fouls_per_90,player_13_avg_fouls_per_90,player_14_avg_fouls_per_90,player_15_avg_fouls_per_90,player_16_avg_fouls_per_90,player_17_avg_fouls_per_90,player_18_avg_fouls_per_90,player_19_avg_fouls_per_90,player_20_avg_fouls_per_90,player_21_avg_fouls_per_90,player_22_avg_fouls_per_90
0,566789,2020-07-06 18:30:00 UTC,"Felix Brych, Germany",78,Bundesliga,180,FC Heidenheim,162,Werder Bremen,16.000000,11.228070,2,2.000000,2,2.122807,4,0.203748,0.039079,0.000000,0.000000,0.234872,0.120277,0.310702,0.138051,0.000000,0.000000,0.000000,0.111973,0.126139,0.164334,0.067861,0.000000,0.258978,0.000000,0.149922,0.000000,0.000000,0.000000,1.165891,0.009770,1.607143,0.000000,0.961507,1.148099,1.165132,1.196441,0.000000,0.957447,1.914894,1.159251,2.711983,1.533780,1.306315,3.829787,0.901243,2.812500,1.452369,1.467391,1.267606,1.725240
1,566788,2020-07-02 18:30:00 UTC,F. Zwayer,78,Bundesliga,162,Werder Bremen,180,FC Heidenheim,11.252941,12.852941,5,2.105882,2,1.794118,7,0.039482,0.000000,0.000000,0.139391,0.069085,0.261811,0.000000,0.254484,0.000000,0.000000,0.112749,0.135034,0.000000,0.141792,0.000000,0.317741,0.000000,0.000000,0.205427,0.115090,0.381282,0.000000,0.009871,0.000000,0.000000,1.208054,1.312608,0.911101,2.000000,1.200848,0.000000,0.000000,1.160648,2.633158,0.000000,1.455729,0.000000,1.191527,2.213115,0.989011,1.175501,2.094629,2.062392,0.000000
2,9822,2017-10-01 10:00:00 UTC,"Alfonso Alvarez Izquierdo, Spain",140,La Liga,548,Real Sociedad,543,Real Betis,7.880952,8.329268,3,2.726190,3,2.890244,6,0.416116,0.152478,0.236894,0.415576,0.195440,0.067214,0.232996,0.357548,0.000000,0.122750,0.325497,0.060376,0.151464,0.158618,0.431530,0.046440,0.226286,0.530105,0.132256,0.077266,0.110810,0.382653,1.961691,0.025413,1.559553,1.440665,0.912052,0.515310,2.139327,1.566402,1.531414,0.036825,2.115732,0.986136,0.878492,0.824815,1.225547,0.789474,1.666286,1.767016,1.697281,0.602679,0.642699,1.454082
3,203592,2016-02-27 21:05:00 UTC,"Alberto Undiano, Spain",140,La Liga,548,Real Sociedad,535,Malaga,0.000000,0.000000,4,3.160000,5,3.250000,9,0.083333,0.185854,0.369155,0.000000,0.170455,0.181818,0.622837,0.340909,0.000000,0.106132,0.390456,0.487952,0.125000,0.504000,0.109890,0.000000,0.552147,0.392157,0.220949,0.000000,0.087891,0.182186,0.000000,1.161590,1.181296,0.000000,0.085227,1.000000,2.179931,1.590909,1.238532,1.167453,1.171367,1.951807,1.000000,1.008000,0.934066,0.602007,1.262051,1.764706,1.104746,0.902461,1.142578,2.004049
4,203530,2016-01-16 21:05:00 UTC,"Juan Martinez Munuera, Spain",140,La Liga,548,Real Sociedad,544,Deportivo La Coruna,0.000000,0.000000,1,3.473684,3,2.631579,4,0.638298,0.289855,0.000000,0.477454,0.191218,0.166256,0.157895,0.171756,0.176471,0.722311,0.203211,0.492341,0.166873,0.608931,0.339196,0.105263,0.120321,0.000000,0.113065,1.065990,0.707865,0.594059,0.957447,1.086957,0.890110,0.477454,1.

In [291]:
!pip install ydata-profiling
from ydata_profiling import ProfileReport

In [292]:
profile = ProfileReport(merged_final, title="Merged final report")
profile

Summarize dataset:   0%|          | 0/5 [00:00<?, ?it/s]


100%|██████████| 60/60 [00:08<00:00,  7.33it/s]


KeyboardInterrupt: 

<Figure size 800x550 with 0 Axes>